# RAG 없이  랭체인 없이  

In [5]:
## 순서대로 
from docx import Document
import tiktoken
import os
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb
from openai import OpenAI

## 1. 문서 읽어오기 #################
document = Document('law_1.docx')

# 읽어온 단락을 하나의 문서로 만들기
full_text = ""

for paragraph in document.paragraphs:
    full_text += f'{paragraph.text}'

## 2. 문서 쪼개기 #################
encoding_model = tiktoken.encoding_for_model('gpt-4o')

# 토큰화
encoding = encoding_model.encode(full_text)

# 함수로 만들기
def split_text(full_text, chunk_size):
    encoding_model=tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoding_model.encode(full_text)
    total_token_count = len(total_encoding)

    test_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding = encoding_model.decode(chunk)
        test_list.append(decoding)

    return test_list

# 청크화 
chunk_list = split_text(full_text, 1500)

## 3. 임베딩 #################
# 오픈AI키 불러오고 임베딩 모델 지정
OPEN_API_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=OPEN_API_KEY, model_name='text-embedding-3-large')

# Chroma 클라이언트 생성하고 이름 지정 
chroma_client = chromadb.Client()

collection_name = 'law_collection'
law_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

# 컬렉션에 설정할 인덱스 생성
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f"{index}")

# 텍스트를 벡터로 변환 + DB 저장
law_collection.add(documents=chunk_list, ids=id_list)
# chromadb에 생성된 컬렉션 목록 확인
collections = chroma_client.list_collections()


## 4. 유사도 검색 #################
query = "전세사기 피해 구제에 대해 알려주세요"

# 유사도 검색
retrieved_docs = law_collection.query(query_texts=query, n_results=3)


# 5. 질문과 전달 #################
# 모델 지정
client = OpenAI()

# 질문과 텍스트 LLM으로 보내기 (페르소나 지정)
response = client.responses.create(
    model="gpt-4o",
    instructions=f'''당신은 전세사기 피해 법률 전문가입니다. 
        [context]를 참고해 사용자의 질문에 답변해주세요.
        
        [context]
        {retrieved_docs['documents'][0]}
        ''',
    input=query,
)

print(response.output_text)

# 질문과 텍스트 LLM으로 보내기 22 

response2 = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "developer",
            "content": f"당신은 전세사기 피해 법률 전문가입니다. {retrieved_docs['documents'][0]}를 참고해 사용자의 질문에 답변해주세요."
        },
        {
            "role": "user",
            "content": query
        }
    ]
)

print(response2.output_text)

전세사기 피해를 구제하기 위한 방법은 다음과 같습니다:

1. **전세사기피해자 인정 신청**: 전세사기피해자법에 따라 피해자로 인정받기 위해 전세사기피해지원위원회의 심의와 국토교통부 장관의 결정을 받아야 합니다.

2. **매각 유예 및 중지 신청**: 지방세 체납으로 인해 압류된 주택이 매각 절차 중일 경우, 지방자치단체장에게 절차의 유예나 중지를 신청할 수 있습니다.

3. **우선매수권**:
   - **경매 절차**: 전세사기피해주택이 경매에 들어갈 경우, 피해자는 보증을 제공하고 최고가격으로 우선매수할 수 있습니다.
   - **공매 절차 (국세 및 지방세)**: 피해자가 공매보증금을 제공하고 최고가(또는 예정가)로 우선매수를 신청할 수 있습니다.

4. **세금 징수의 특례**:
   - **국세 및 지방세 안분 징수**: 피해자로 인정된 경우, 임대인이 여러 주택을 소유하고 있을 때 임대인의 세금을 안분하여 징수할 수 있습니다. 이를 통해 피해자의 보증금 반환 우선권을 보호합니다.

이 과정에서 필요한 서류와 절차는 관련 법률에 따라 진행되므로, 각 단계에서 전문가의 도움을 받는 것이 좋습니다.
전세사기 피해자를 위한 법적 구제 방안은 다음과 같은 절차와 혜택으로 구성됩니다:

1. **전세사기피해자 인정**
   - 피해자는 먼저 전세사기피해자로 인정받아야 합니다. 이를 위해 전세사기피해자 관련 요건을 충족하고 전세사기피해지원위원회의 심의를 통해 국토교통부장관의 결정을 받아야 합니다.

2. **경매 및 공매 절차 지원**
   - 전세사기 피해자는 주택이 경매나 공매에 넘어갈 경우 우선매수권을 행사하여 경매 절차에서 주택을 매수할 수 있습니다.
   - 매각 유예나 중지 요청을 통해 시급한 매각 절차를 일시적으로 중단시킬 수도 있습니다.

3. **세금 관련 특례**
   - 전세사기 피해자들은 국세 또는 지방세의 우선 징수에 대한 특례를 받을 수 있습니다. 이는 임대인의 여러 주택에 걸쳐 세금을 안분하여 부과함으로써 임차인의 보증금 반환에 